# SQL Assignment - Complete Notebook
This notebook includes all 10 questions with theoretical answers and practical SQL examples using SQLite.

In [ ]:

# SQL Assignment Notebook - Complete Ready-to-Run

# Import sqlite3
import sqlite3

# Q1-Q5 will be included first as per previous plan

# Q1 Example: DDL, DML, DQL
with sqlite3.connect('assignment.db') as conn:
    cursor = conn.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS students (
                        id INTEGER PRIMARY KEY,
                        name TEXT,
                        marks INTEGER
                      )''')
    cursor.execute("INSERT OR IGNORE INTO students (name, marks) VALUES ('Alice', 85)")
    cursor.execute("INSERT OR IGNORE INTO students (name, marks) VALUES ('Bob', 90)")
    cursor.execute("SELECT * FROM students")
    rows = cursor.fetchall()
    for row in rows:
        print(row)

# Q2 Example: Constraints
with sqlite3.connect('assignment.db') as conn:
    cursor = conn.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS users (
        user_id INTEGER PRIMARY KEY,
        username TEXT NOT NULL UNIQUE,
        email TEXT UNIQUE
    )''')
    cursor.execute("INSERT OR IGNORE INTO users (username, email) VALUES ('user1', 'a@example.com')")
    cursor.execute("INSERT OR IGNORE INTO users (username, email) VALUES ('user2', 'b@example.com')")
    cursor.execute("SELECT * FROM users")
    print(cursor.fetchall())

# Q3 Example: LIMIT and OFFSET
with sqlite3.connect('assignment.db') as conn:
    cursor = conn.cursor()
    cursor.execute("DELETE FROM students")
    for i in range(1, 31):
        cursor.execute(f"INSERT INTO students (name, marks) VALUES ('Student{i}', {50+i})")
    cursor.execute("SELECT * FROM students LIMIT 10 OFFSET 20")
    for row in cursor.fetchall():
        print(row)

# Q4 Example: CTE
with sqlite3.connect('assignment.db') as conn:
    cursor = conn.cursor()
    cursor.execute('''WITH HighMarks AS (
        SELECT name, marks FROM students WHERE marks > 75
    )
    SELECT * FROM HighMarks''')
    for row in cursor.fetchall():
        print(row)

# Q5 Example: Normalization
with sqlite3.connect('assignment.db') as conn:
    cursor = conn.cursor()
    cursor.execute("CREATE TABLE IF NOT EXISTS student_phones (student_id INTEGER, phone TEXT)")
    cursor.execute("INSERT OR IGNORE INTO student_phones (student_id, phone) VALUES (1, '1234567890')")
    cursor.execute("INSERT OR IGNORE INTO student_phones (student_id, phone) VALUES (1, '0987654321')")
    cursor.execute("SELECT * FROM student_phones")
    print(cursor.fetchall())

# Q6: ECommerceDB setup
with sqlite3.connect('ECommerceDB.db') as conn:
    cursor = conn.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS Categories (
        CategoryID INTEGER PRIMARY KEY,
        CategoryName TEXT NOT NULL UNIQUE
    )''')
    cursor.execute('''CREATE TABLE IF NOT EXISTS Products (
        ProductID INTEGER PRIMARY KEY,
        ProductName TEXT NOT NULL UNIQUE,
        CategoryID INTEGER,
        Price REAL NOT NULL,
        StockQuantity INTEGER,
        FOREIGN KEY(CategoryID) REFERENCES Categories(CategoryID)
    )''')
    cursor.execute('''CREATE TABLE IF NOT EXISTS Customers (
        CustomerID INTEGER PRIMARY KEY,
        CustomerName TEXT NOT NULL,
        Email TEXT UNIQUE,
        JoinDate TEXT
    )''')
    cursor.execute('''CREATE TABLE IF NOT EXISTS Orders (
        OrderID INTEGER PRIMARY KEY,
        CustomerID INTEGER,
        OrderDate TEXT NOT NULL,
        TotalAmount REAL,
        FOREIGN KEY(CustomerID) REFERENCES Customers(CustomerID)
    )''')
    categories = [(1, 'Electronics'), (2, 'Books'), (3, 'Home Goods'), (4, 'Apparel')]
    cursor.executemany("INSERT OR IGNORE INTO Categories VALUES (?, ?)", categories)
    products = [
        (101, 'Laptop Pro', 1, 1200.00, 50),
        (102, 'SQL Handbook', 2, 45.50, 200),
        (103, 'Smart Speaker', 1, 99.99, 150),
        (104, 'Coffee Maker', 3, 75.00, 80),
        (105, 'Novel: The Great SQL', 2, 25.00, 120),
        (106, 'Wireless Earbuds', 1, 150.00, 100),
        (107, 'Blender X', 3, 120.00, 60),
        (108, 'T-Shirt Casual', 4, 20.00, 300)
    ]
    cursor.executemany("INSERT OR IGNORE INTO Products VALUES (?, ?, ?, ?, ?)", products)
    customers = [
        (1, 'Alice Wonderland', 'alice@example.com', '2023-01-10'),
        (2, 'Bob the Builder', 'bob@example.com', '2022-11-25'),
        (3, 'Charlie Chaplin', 'charlie@example.com', '2023-03-01'),
        (4, 'Diana Prince', 'diana@example.com', '2021-04-26')
    ]
    cursor.executemany("INSERT OR IGNORE INTO Customers VALUES (?, ?, ?, ?)", customers)
    orders = [
        (1001, 1, '2023-04-26', 1245.50),
        (1002, 2, '2023-10-12', 99.99),
        (1003, 3, '2023-07-01', 145.00),
        (1004, 3, '2023-01-14', 150.00),
        (1005, 4, '2023-09-24', 120.00),
        (1006, 1, '2023-06-19', 20.00)
    ]
    cursor.executemany("INSERT OR IGNORE INTO Orders VALUES (?, ?, ?, ?)", orders)
    cursor.execute("SELECT * FROM Orders")
    for row in cursor.fetchall():
        print(row)

# Q7: Total orders per customer
with sqlite3.connect('ECommerceDB.db') as conn:
    cursor = conn.cursor()
    cursor.execute('''
    SELECT C.CustomerName, C.Email, 
           COUNT(O.OrderID) AS TotalNumberofOrders
    FROM Customers C
    LEFT JOIN Orders O ON C.CustomerID = O.CustomerID
    GROUP BY C.CustomerID
    ORDER BY C.CustomerName
    ''')
    for row in cursor.fetchall():
        print(row)

# Q8: Product info with category
with sqlite3.connect('ECommerceDB.db') as conn:
    cursor = conn.cursor()
    cursor.execute('''
    SELECT P.ProductName, P.Price, P.StockQuantity, C.CategoryName
    FROM Products P
    JOIN Categories C ON P.CategoryID = C.CategoryID
    ORDER BY C.CategoryName, P.ProductName
    ''')
    for row in cursor.fetchall():
        print(row)

# Q9: Top 2 most expensive products per category using CTE
with sqlite3.connect('ECommerceDB.db') as conn:
    cursor = conn.cursor()
    cursor.execute('''
    WITH RankedProducts AS (
        SELECT P.ProductName, P.Price, C.CategoryName,
               ROW_NUMBER() OVER (PARTITION BY C.CategoryName ORDER BY P.Price DESC) AS rn
        FROM Products P
        JOIN Categories C ON P.CategoryID = C.CategoryID
    )
    SELECT ProductName, Price, CategoryName
    FROM RankedProducts
    WHERE rn <= 2
    ''')
    for row in cursor.fetchall():
        print(row)

# Q10: Sakila simulation (top customers example)
with sqlite3.connect('SakilaSim.db') as conn:
    cursor = conn.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS Customers (
        CustomerID INTEGER PRIMARY KEY,
        CustomerName TEXT,
        Email TEXT
    )''')
    cursor.execute('''CREATE TABLE IF NOT EXISTS Rentals (
        RentalID INTEGER PRIMARY KEY,
        CustomerID INTEGER,
        FilmID INTEGER,
        RentalDate TEXT,
        Amount REAL,
        FOREIGN KEY(CustomerID) REFERENCES Customers(CustomerID)
    )''')
    cursor.execute('''CREATE TABLE IF NOT EXISTS Films (
        FilmID INTEGER PRIMARY KEY,
        FilmName TEXT,
        CategoryName TEXT
    )''')
    # Example top 5 customers query
    cursor.execute('''
    SELECT CustomerName, Email, SUM(Amount) AS TotalSpent
    FROM Rentals R
    JOIN Customers C ON R.CustomerID = C.CustomerID
    GROUP BY R.CustomerID
    ORDER BY TotalSpent DESC
    LIMIT 5
    ''')
